# Example notebook

In [ ]:
%matplotlib inline
from skimage import transform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.functional as F
import cv2
import os
import time
import glob
import PIL.Image
import io
import shutil
from torch.utils.data import DataLoader
from floortrans.models import get_model
from floortrans.loaders import FloorplanSVG, DictToTensor, Compose, RotateNTurns
from floortrans.plotting import segmentation_plot, polygons_to_image, draw_junction_from_dict, discrete_cmap
discrete_cmap()
from floortrans.post_prosessing import split_prediction, get_polygons, split_validation
from mpl_toolkits.axes_grid1 import AxesGrid
from scipy import misc

rot = RotateNTurns()
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]

size = 512
resize = size, size
data_folder = 'data/cubicasa5k/'
data_file = 'train.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
n_rooms = 12
n_icons = 11
split = [21, 12, 11]
model.conv4_ = torch.nn.Conv2d(256, n_classes, bias=True, kernel_size=1)
model.upsample = torch.nn.ConvTranspose2d(n_classes, n_classes, kernel_size=4, stride=4)
checkpoint = torch.load('model_best_val_loss_var.pkl')

model.load_state_dict(checkpoint['model_state'])
model.eval()
model.cuda()
print("Model loaded!")

save_folder = 'E:/DataSets/cubicasa5kv1.1/'
i = 1

for item in data_iter:
    val = item
    junctions = val['heatmaps']
    folder = val['folder'][0]
    image = val['image'].cuda()
    label = val['label']

    label_np = label.data.numpy()[0]
    
    height = label_np.shape[1]
    width = label_np.shape[2]
    img_size = (height, width)
    
    x_size = width/height*10
    y_size = 10
    
    fig = plt.figure(frameon=False)
    fig.set_size_inches(x_size, y_size, forward=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.axis('off')
    fig.add_axes(ax)
    rseg = ax.imshow(label_np[0], cmap='rooms', vmin=0, vmax=n_rooms-0.1)
    
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    
    im = PIL.Image.open(buf)
    
    # generating the thumbnail from given size
    im.thumbnail(resize, PIL.Image.ANTIALIAS)

    offset_x = max((resize[0] - im.size[0]) / 2, 0)
    offset_y = max((resize[1] - im.size[1]) / 2, 0)

    offset_tuple = (int(offset_x), int(offset_y)) #pack x and y into a tuple
    
    # create background to paste onto so it is black
    im_bg = PIL.Image.new(mode='RGBA',size=resize,color=(0, 0, 0, 255))

    # Pasting onto bg
    im_bg.paste(im, offset_tuple, im)
    
    # save (the PNG format will retain the alpha band unlike JPEG)
    im_bg.save('E:/DataSets/cubicasa5kv1.1/' + str(i) + '/function_map_no_icon.png','PNG')
    
    buf.close()
    
    print('Completed ' + str(i))
    
    i += 1    